<a href="https://colab.research.google.com/github/rimbourouphael/NSY107/blob/main/Model-1_Original_Imbalanced_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Deep Graph Convolutional Neural Network (DGCNN)

This code is part our research on malware detection and classification using Deep Learning and Deep Graph Convolutional Neural Networks.

For more information or citation, please refer to our research paper:

"Oliveira, Angelo; Sassi, Renato José (2019): Behavioral Malware Detection Using Deep Graph Convolutional Neural Networks. TechRxiv. Preprint." at https://doi.org/10.36227/techrxiv.10043099.v1

For the dataset, please refer to our repository:

https://ieee-dataport.org/open-access/malware-analysis-datasets-api-call-sequences

#### Model-1, Original (imbalanced) Dataset

In [1]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 155 kB 5.0 MB/s 


In [2]:
import numpy as np
SEED = 137
np.random.seed(SEED)

import time
from collections import Counter

import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import precision_score, f1_score, roc_auc_score

from imblearn.pipeline import Pipeline

import torch
torch.manual_seed(SEED)
import torch.nn as nn
import torch.nn.functional as F

from skorch.classifier import NeuralNetBinaryClassifier

### Data Preprocessing

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/rimbourouphael/NSY107/main/dynamic_api_call_sequence_per_malware_100_0_306.csv')
df.head()

,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,malware
0,071e8c3f8922e186e57548cd4c703a5d,112,274,158,215,274,158,215,298,76,...,71,297,135,171,215,35,208,56,71,1
1,33f8e6d08a6aae939f25a8e0d63dd523,82,208,187,208,172,117,172,117,172,...,81,240,117,71,297,135,171,215,35,1
2,b68abd064e975e1c6d5f25e748663076,16,110,240,117,240,117,240,117,240,...,65,112,123,65,112,123,65,113,112,1
3,72049be7bd30ea61297ea624ae198067,82,208,187,208,172,117,172,117,172,...,208,302,208,302,187,208,302,228,302,1
4,c9b3700a77facf29172f32df6bc77f48,82,240,117,240,117,240,117,240,117,...,209,260,40,209,260,141,260,141,260,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43876 entries, 0 to 43875
Columns: 102 entries, hash to malware
dtypes: int64(101), object(1)
memory usage: 34.1+ MB


In [5]:
X = df.drop(['hash', 'malware'], axis = 1).values.astype(int)
y = df['malware'].values.astype(int)
print(X.shape)
print(y.shape)

(43876, 100)
(43876,)


In [6]:
print(X.min())
print(X.max())

0
306


In [7]:
def check_imbalance(dataset):
    count = sorted(Counter(dataset).items())
    print(count)
    print(count[1][1] / count[0][1])
    return

In [8]:
check_imbalance(y)

[(0, 1079), (1, 42797)]
39.66357738646895


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED)

In [10]:
check_imbalance(y_train)
check_imbalance(y_test)

del df, X, y

[(0, 731), (1, 29982)]
41.015047879616965
[(0, 348), (1, 12815)]
36.82471264367816


### Architecture

In [11]:
def norn_adj(X, input_dim_1):
    
    A = torch.zeros((X.size(0), input_dim_1, input_dim_1), dtype = torch.float).cuda()
        
    A_view = A.view(A.size(0), -1)
    x_size = X.size(-1)
    indices = X.narrow(-1, 0, x_size - 1) * A.stride(1) * A.stride(2) + X.narrow(-1, 1, x_size - 1) * A.stride(2)
    A_view.scatter_(1, indices, 1)
        
    A_hat = A + torch.eye(input_dim_1, dtype = torch.float).cuda()
    D_hat = A_hat.sum(dim = 1).pow(-1.0).diag_embed()
    
    return A_hat, D_hat

def to_one_hot(X, input_dim_1):
    
    X = F.one_hot(X, num_classes = input_dim_1).float()    
    X = X.permute(0, 2, 1)
    
    return X

class DGCNN_network(nn.Module):
    
    def __init__(self, weight_dim_1, weight_dim_2):

        super(DGCNN_network, self).__init__()
        self.weight_dim_1 = weight_dim_1
        self.weight_dim_2 = weight_dim_2
        self.weights = nn.Parameter(torch.rand((self.weight_dim_1, weight_dim_2), dtype = torch.float, requires_grad = True))
        
    def forward(self, A_hat, D_hat, X):
        return D_hat.matmul(A_hat).matmul(X).matmul(self.weights)

class Model_1_network(nn.Module):
    
    def __init__(self, input_dim_1, input_dim_2, weight_dim_2, dropout_rate):
        
        super(Model_1_network, self).__init__()
        
        self.input_dim_1 = input_dim_1
        self.input_dim_2 = input_dim_2
        self.weight_dim_1 = input_dim_2
        self.weight_dim_2 = weight_dim_2
        self.dropout_rate = dropout_rate
        
        self.dgcnn = DGCNN_network(self.weight_dim_1, self.weight_dim_2)
        self.dropout = nn.Dropout(p = self.dropout_rate)
        self.fc = nn.Linear(self.input_dim_1 * self.weight_dim_2, 1)
        
    def forward(self, X):
        
        A_hat, D_hat = norn_adj(X, input_dim_1)
        X = to_one_hot(X, input_dim_1)        

        H = self.dgcnn(A_hat, D_hat, X)
        H = self.dropout(H)
        H = torch.relu(H)
        H = H.view(H.size(0), -1)
        H = self.fc(H)
                
        return H.squeeze()
    
model = Model_1_network(
    input_dim_1 = 307,
    input_dim_2 = 100,
    weight_dim_2 = 31,
    dropout_rate = 0.4
)

print(model)
print(f'\nParameters: {np.sum([param.numel() for param in model.parameters()])}')
del model    

Model_1_network(
  (dgcnn): DGCNN_network()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=9517, out_features=1, bias=True)
)

Parameters: 12618


### Hyperparameter Optimization

In [ ]:
start_training = time.time()

torch.manual_seed(SEED)

input_dim_1 = 307
input_dim_2 = 100

net = NeuralNetBinaryClassifier(
    Model_1_network,
    module__input_dim_1 = input_dim_1,
    module__input_dim_2 = input_dim_2,
    module__weight_dim_2 = 31,
    module__dropout_rate = 0.4,
    batch_size = 32,
    max_epochs = 10,
    train_split = None,
    optimizer = torch.optim.Adam,
    device = 'cuda'
)

pipe = Pipeline([
    ('net', net)
])

# LSTM(10) => 12771 => DGCNN(31) => 12618
# LSTM(40) => 55881 => DGCNN(137) => 55760
# LSTM(70) => 106191 => DGCNN(261) => 106228 
# LSTM(100) => 163701 => DGCNN(402) => 163615

params = {
    'net__module__weight_dim_2' : [31, 137, 261, 402],
    'net__module__dropout_rate' : [0.4, 0.5, 0.6],
    'net__batch_size' : [32, 64, 128],
    'net__max_epochs' : [10, 20, 30]
}

gs = GridSearchCV(
    pipe,
    params,
    refit = False,
    cv = StratifiedKFold(n_splits = 5, random_state = SEED, shuffle = True),
    scoring = lambda net_gs, X_gs, y_gs : roc_auc_score(y_gs, net_gs.predict_proba(X_gs)),
    verbose = 2
)

gs.fit(X_train, y_train.astype(np.float))

print(gs.best_score_, gs.best_params_)

end_training = (time.time() - start_training) / 60

print(f'\nDone! Time: {end_training:.2f} min')

#### Best Configuration:

0.965135479422108 {'net__batch_size': 128, 'net__max_epochs': 10, 'net__module__dropout_rate': 0.6, 'net__module__weight_dim_2': 31}

### Training

In [12]:
torch.manual_seed(SEED)

# weight_dim_2 = gs.best_params_['net__module__weight_dim_2']
# dropout_rate = gs.best_params_['net__module__dropout_rate']
# batch_size = gs.best_params_['net__batch_size']
# max_epochs = gs.best_params_['net__max_epochs']

input_dim_1 = 307
input_dim_2 = 100
weight_dim_2 = 31
dropout_rate = 0.6
batch_size = 128
max_epochs = 10

net = NeuralNetBinaryClassifier(
    Model_1_network,
    module__input_dim_1 = input_dim_1,
    module__input_dim_2 = input_dim_2,
    module__weight_dim_2 = weight_dim_2,
    module__dropout_rate = dropout_rate,
    batch_size = batch_size,
    max_epochs = max_epochs,
    train_split = None,
    optimizer = torch.optim.Adam,
    iterator_train__shuffle = True,
    device = 'cuda'
)

pipe = Pipeline([
    ('net', net)
])

pipe.fit(X_train, y_train.astype(np.float))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


  epoch    train_loss     dur
-------  ------------  ------
      1        0.1687  7.9533
      2        0.0455  6.5472
      3        0.0369  6.5051
      4        0.0311  6.5606
      5        0.0288  6.4991
      6        0.0248  6.5136
      7        0.0288  6.4699
      8        0.0237  6.5957
      9        0.0244  6.5458
     10        0.0241  6.4982


Pipeline(steps=[('net',
                 <class 'skorch.classifier.NeuralNetBinaryClassifier'>[initialized](
  module_=Model_1_network(
    (dgcnn): DGCNN_network()
    (dropout): Dropout(p=0.6, inplace=False)
    (fc): Linear(in_features=9517, out_features=1, bias=True)
  ),
))])

### Evaluation

In [13]:
def model_evaluate(y, pred):
    print('Confusion matrix\n[TN FP]\n[FN TP]')
    print(confusion_matrix(y >= 0.5, pred >= 0.5))
    print(f'Accuracy: {accuracy_score(y >= 0.5, pred >= 0.5):.4f}')
    print(f'Precision: {precision_score(y >= 0.5, pred >= 0.5):.4f}')
    print(f'Recall: {recall_score(y >= 0.5, pred >= 0.5):.4f}')
    print(f'F1-Score: {f1_score(y >= 0.5, pred >= 0.5):.4f}')
    print(f'ROC AUC: {roc_auc_score(y, pred):.4f}')
    return

In [14]:
model_evaluate(y_test, np.ones(len(y_test)))

Confusion matrix
[TN FP]
[FN TP]
[[    0   348]
 [    0 12815]]
Accuracy: 0.9736
Precision: 0.9736
Recall: 1.0000
F1-Score: 0.9866
ROC AUC: 0.5000


In [15]:
X_test_1 = pipe.predict_proba(X_test)[:, 1]
model_evaluate(y_test, X_test_1)

Confusion matrix
[TN FP]
[FN TP]
[[  217   131]
 [   15 12800]]
Accuracy: 0.9889
Precision: 0.9899
Recall: 0.9988
F1-Score: 0.9943
ROC AUC: 0.9753


In [ ]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>